# Innodb核心功能

### 内存池：
* 所有的更新或者写操作最先发生在内存中，然后以一定的频率刷新到磁盘
* 缓冲池：
    * 按页(16KB)读取文件到缓存池中，按最近最少使用(LRU)算法更新缓存池中的数据
    * 缓存的数据项包括：索引页，数据页，undo页，插入缓存，自适应哈希索引，innodb存储的锁信息，数据字典信息
* 重做日志缓存池：主要负责记录重做日志信息，每秒钟刷新到磁盘，即使事务没有提交

### 主线程循环逻辑：
* 每一秒钟循环操作内容：
    * 刷新重做日志到磁盘，即使事务没提交
    * 可能合并插入缓存，当前1秒io次数少于5次时执行
    * 可能刷新最多100个缓存池中的脏页到磁盘，当脏页超过最大脏页比时执行
    * 可能切换到后台循环线程
* 每十秒循环操作内容：
    * 刷新重做日志到磁盘
    * 可能刷新100个脏页数据，过去10秒io次数少于200次
    * 合并插入缓存
    * 刷新100个或10个脏页数据
    * 删除无用undo页
    * 产生检查点
* 后台循环：
    * 删除无用undo页
    * 合并插入缓存
    * 跳回主循环线程
    * 可能跳转到刷新线程
* 刷新循环：如果无其他任务，则调整到挂起循环
* 挂起循环：将主循环挂起

### 关键特性：
* 支持行锁，外键，非锁定读
* 实现默认四种隔离级别，通过next-key locking策略避免幻读
* 通过MVCC控制高并发
* 插入缓存，提高修改性能：如果当前缓存池中包含即将插入的非聚集索引页则直接插入，否则写入到插入缓冲区中，然后在以一定的频率合并到非聚餐索引叶子节点（刷入磁盘）
* 两次写，提高可靠性：刷新脏页时，会将脏页数据拷贝到doublewrite缓存中，然后在将数据写入到共享表空间，然后在将数据写入到各个表空间，这样在数据恢复时可以在共享表空间找到对应的副本并利用重做日志还原数据
* 自适应HASH索引，提高查询性能：根据访问频率建立HASH索引，只能用于等值查询
* 预读取：
    * 随机预读取，当缓存中存在一个区的13个页并且在LRU列表的前端，则会将该区中的所有页都读入缓存，已取消
    * 线性预读取，当一个区中的24个页都被顺序访问过，则会下一个区的所有页
* 按聚集索引方式存储数据

### 文件
* 表空间文件：
    * 存储数据库表的所有信息，分为独立的表空间文件和共享表空间文件，如果存在独立表空间，则独立表空间负责数据、索引和插入缓存，共享表空间复制存储undo信息，事务信息，二次写缓冲等；否则所有数据都被逻辑存储在共享表空间中
    * ```innodb_data_file_path```：指定innodb表空间文件存储路径，默认所有表共享该空间，文件结尾后缀为ibdataXXX
    * ```innodb_file_per_table```：指定每个表使用独立的表空间文件，文件后缀为ibdXXX
* 表结构文件：定义表的结构，与存储引擎无关，以后缀frm结尾
* 重做日志文件：
    * 记录innodb的事务日志，用于断电后故障恢复，文件名为ib_logfileXXX
    * 采取循环写覆盖写
    * 记录innodb本身物理页的改变，事务进行过程中也会记录
    * ```innodb_log_file_size```：指定重做日志文件大小
    * ```innodb_log_files_in_group```：指定日志文件组中重做日志的数量
    * ```innodb_mirrored_log_groups```：指定日志镜像文件组，即日志文件组的数量
    * ```innodb_log_group_home_dir```：指定日志文件组路径
    * ```innodb_flush_log_at_trx_commit```：设置重做日志缓存刷新规则
        * 0表示不根据commit提交，只根据时间刷新
        * 1表示commit时刷新
        * 2表示异步刷新也就是说commit的时候不一定会刷新只会有这么一个动作
        
### 逻辑存储
* 表空间由段、区、页组成
    <img src='../images/mysql/存储引擎存储空间.png' width='400px'>
    * 段由数据段，索引段，回滚段组成，数据段为B+树的叶子节点，索引段为B+树的非叶子节点，大的数据段每次最多可以申请4个区
    * 区由64个连续的页组成，每个页大小16KB，每个区1MB，页是最小的存储单位
    * 页包含数据页，索引页，undo页，系统页，事务数据页，插入缓存位图页，插入缓存空闲列表页，未压缩的二进制大对象页，压缩的二进制大对象页
        <img src='../images/mysql/存储引擎页结构.png' width='200px'>
    * 行数据存储在页中的User Records中，页中包含多行数据，最多包含7992行数据
        * Compact行格式：
            <img src='../images/mysql/存储引擎行结构1.png' width='400px'>
            * 变长字段长度列表记录所有变长字段列使用的长度，大小不能超过2字节，因此每行数据中所有varchar类型的列不超过65535字节，并且varchar(N)中的N是指字符数量
            * NULL标志位记录行数据中是否包含NULL值，varchar和char类型为NULL值的列不占用存储空间
            * 为了让数据页至少存放两行数据，因此存储varchar类型超过8098字节时，会采取溢出的形式存储，前768字节存储在数据页中，剩下的存储到未压缩的二进制大对象页中
            * BLOB和TEXT类型的数据存储与varchar类似
            * 每行包含事务ID列（6字节）和回滚指针列（7字节）两个隐藏列
            * 如果没有显示定义主键，会增加一个6字节的RowID列作为主键
        * Redundant行格式
            <img src='../images/mysql/存储引擎行结构2.png' width='400px'>   
            * 对于varchar类型的NULL值不占用空间，char类型的NULL需要占用存储空间
        * Compressed和Dynamic行格式
            * BLOB和TEXT类型的数据只存储前20字节指针，实际数据都存储在BLOB页中
            * Compressed行格式会对BLOB, TEXT, varchar类型溢出的数据进行压缩

### 数据完整性
* 实体完整性：保证表中有一个主键
* 域（列）完整性：保证数据的值满足特定的条件
* 参照完整性：保证两张表之间的关系
* 通过约束来实现数据的完整性，支持的约束：
    * Primary Key约束
    * Foreign Key约束：可以指定DELETE和UPDATE的级联操作，及时触发
        * CASCADE：当父表发生DELETE或UPDATE操作时，子表也会被DELETE或UPDATE
        * SET NULL：当父表发生DELETE或UPDATE操作时，子表会将对应列更新为NULL
        * NO ACTION：当父表发生DELETE或UPDATE操作时，抛出错误，不允许这类操作
        * RESTRICT：当父表发生DELETE或UPDATE操作时，抛出错误，不允许这类操作，外键约束级联操作的默认值
    * Unique Key约束
    * NOT NULL
    * Default
    * ENUM约束
    * SET约束
    * 触发器
        * 在数据更新，插入，删除之前或之后做约束检查，提供高级的完整性约束检查

### 数据索引，提速查询
* B+树索引只能找到对应的数据页，然后将数据页加载到内存中查找具体的行数据
* 聚集索引：
    * 按照每张表的主键构造一颗B+树，并且在叶节点中存放整张表的行记录数据，因此叶节点也是数据页，每个数据页通过一个双向链表链接
    * 一张表只能建立一个聚集索引
    * 数据页存放完整的行记录
    * 索引页存放键值以及指向数据页的偏移量
    * 数据页按逻辑顺序存储，物理存储是不连续的
    * 聚集索引对于主键的排序查找和范围查找非常快，因为叶子节点通过双向链表链接，可以快速定位主键；其次索引页记录了叶子节点的范围，可以快速找出范围内的数据
    * 聚集索引的创建：创建临时表，把数据导入临时表，删除原表，将临时表重命名为原表
    * 查询数据时，通过叶节点指针顺序读取数据
* 非聚集索引（辅助索引）：
    * 叶节点包含对应的键值以及指向聚集索引主键的指针
    * 非聚集索引查找过程：先通过键值找到非聚集索引的叶子节点，然后通过指向聚集索引主键的指针在聚集索引中查找对应的叶节点，最后取出行数据
    * 一张表可以有多个非聚集索引
    * 非聚集索引的创建：创建过程中，对表加S锁，只能读不能写，不需要重建表
    * 非聚集索引的删除：直接将非聚集索引占用的空间标记为可用
    * 查询数据时，通过叶节点的主键指针随机读取数据
* B+树索引使用情况分析（何时建立非聚集索引）
    * 低选择性意味着列值可选范围很小，数据多，如性别，不推荐使用
    * 高选择性意味着列值可选范围很大，数据少，几乎没重复性，例如姓名，推荐使用
    * 但是在高选择性列上查询大范围数据时（全部数据的20%），查询优化器会直接使用全表扫描
* 联合索引
    * 对多个列建立索引，节点中的key是一个多元组，元组值顺序按索引定义时的列顺序
    * 假如联合索引是(a, b)，则```WHERE a=XXX AND b=XXX; WHERE a=XXX;```都会使用索引，而```WHERE b=XXX;```则不会
* 自适应HASH索引：只能用于等值比较，且采取链表方式解决HASH冲突

### 数据锁，保证并发一致性，实现事务隔离性
* 标准的行级锁类型：
    * 共享锁（S Lock），允许事务读一行数据，同时允许多个事务获取共享锁读取数据
    * 排他锁（X Lock），允许事物删除或更新一行数据
* 意向锁类型，表级锁：为了在一个事务中请示下一行将被请求的锁类型
    * 意向共享锁（IS Lock），事务想获得一个表中某几行的共享锁
    * 意向排他锁（IX Lock），事务想获得一个表中某几行的排他锁
    * 意向锁不会阻塞除全表扫描以外的任何请求
* 查看Innodb当前状态下锁的状态：
    * information_schema.innodb_trx：当前事务的状态信息，分析锁资源使用情况
    * information_schema.innodb_locks：直接查看锁资源的状态信息
    * information_schema.innodb_lock_waits：查看等待锁资源的事务ID信息
* 一致性的非锁定读，MVCC，默认的读方式
    * 读数据的同时其他事务在执行DELETE或UPDATE操作，读取操作不会被阻塞，会读取行数据的一个版本快照，无需等待X锁的释放
    * 版本快照通过undo页实现，每次更新操作会将历史数据存储到undo页中，并在新数据中添加一个指向该undo页的指针
    * READ CIMMITED和REPEATABLE READ事务隔离级别使用一致性的非锁定读
        * READ COMMITED：会读取被锁定行的最新快照版本
        * REPEATABLE READ：会读取事务开始时被锁定行的快照版本
    * 工作原理：
        * 注意，这里所说的时间并不是真实的系统时间，而是事务序列号（系统版本号），每开启一个事务，该号会自动自增
        * 建立两个隐藏列存储行数据的创建时间和行数据的过期时间（删除时间）
        * SELECT操作：要求行数据的创建时间必须早于当前事务的时间并且行数据的删除时间要么未定义要么要晚于当前事务的时间
        * INSERT操作：将当前事务的时间作为新插入的行数据版本号
        * DELETE操作：将当前事务的时间添加为行数据的过期时间
        * UPDATE操作：将当前事务的时间添加为新行数据的创建时间，同时将当前事务的时间添加为旧的行数据的过期时间
* 加锁读：
    * SELECT ... FOR UPDATE：对读取的数据加X锁，会阻塞其他事务在该数据上加其他锁，但是不会阻塞一致性非锁定读的操作
    * SELECT ... LOCK IN SHARE MODE：对读取的数据加S锁，会阻塞其他事务在该数据上加X锁
    * 使用上述两种操作时务必关闭事务自动提交功能，例如加上BEGIN或START TRANSACTION或SET AUTOCOMMIT=0操作
* 自增长主键插入的问题：
    * 自增长主键插入时，会使用AUTO-INC Locking锁机制，该锁机制不是在事务提交时释放，而是SQL语句操作完成后立即释放
    * 大批量插入数据时，并发性能会下降，因为后面的插入操作必须等待前面的插入操作完成
    * 大批量插入数据时，会阻塞其他事务中的插入操作
    * 新版本有AUTO-INC Locking锁机制和通过互斥量累加计数器两种操作模式，可通过参数innodb_aotu_lock_mode进行配置
* 外键和锁
    * 如果没有给外键列添加索引，则会自动给外键列添加一个索引，避免锁表
    * 对于外键的插入或更新，需要先SELECT父表，这时使用的是SELECT ... LOCK IN SHARE MODE模式读取数据，保证数据一致性
    
### 事务，保证数据库ACID
* 原子性、一致性、持久性通过redo和undo实现
    * REDO：事务开始时，将该事物的日志序列号（LSN）记录入重做日志的缓存；事务执行时，将事务的操作记录到重做日志缓存；事务提交时，将重做日志缓存写入磁盘，即写数据前先写日志
    * UNDO：对于数据库的修改，不仅记录重做日志，而且还会记录undo信息，undo信息存储在共享表空间中的undo段内，undo信息其实就是事务的反向操作
* 事务控制语句：
    * START TRANSACTION|BEGIN：存储过程中只能使用START TRANSACTION显示开启一段事务操作
    * COMMIT|COMMIT WORK & ROLLBACK|ROLLBACK WORK：
        * COMMIT WORK控制事务提交后的行为，默认情况下与COMMIT行为一致，可通过completion_type设置
        * ROLLBACK WORK与COMMIT WORK概念一致
    * SAVEPOINT identifier & RELEASE SAVEPOINT identifier & ROLLBACK TO [SAVEPOINT] identifier：
        * ROLLBACK TO [SAVEPOINT] identifier并不代表事务结束，还是需要显示的调用COMMIT|ROLLBACK操作结束事务
    * SET TRANSACTION
* 事务的隔离级别
    * READ UNCOMMITTED
    * READ COMMITTED
    * REPEATABLE READ：默认隔离级别，通过Next-Key Lock锁算法，锁定行以及范围内的数据，确保不会出现幻读现象，达到其他数据库SERIALIZABLE的隔离级别
    * SERIALIZABLE：会给每个SELECT语句添加LOCK IN SHARE MODE模式，给读操作添加共享锁，主要用于InnoDB存储引擎的分布式事务
* XA事务支持分布式事务，XA事务运行不同数据库之间的分布式事务，只要每个参与节点都支持XA事务
    * XA事务是存储引擎提供的支持，因此不同存储引擎之间的事务也会通过XA事务协调
    * 处理分布式事务时，隔离级别必须设置为SERIALIZABLE


****

# Innodb系统参数
* innodb_read_io_threads，innodb_write_io_threads：指定读写线程数量
* innodb_buffer_pool_size：指定缓冲池大小
* innodb_log_pool_size：指定重做日志缓冲池大小
* innodb_additianal_mem_pool_size：指定额外的内存池大小
* innodb_max_dirty_pages_pct：最大脏页比，脏页超过这个比值就会触发脏页刷盘
* innodb_io_capacity：表示磁盘IO吞吐量
    * 在合并插入缓存时，选择该值的5%
    * 刷新脏页罗盘时，刷新脏页的数量由该值指定
* innodb_adaptive_flushing：自适应刷新，自动判断刷新脏页的数量
* skip_innodb_doublewrite：设置两次写功能
* innodb_adaptive_hash_index：设置自适应HASH索引
* innodb_fast_shutdown：设置innodb关闭时执行的操作
    * 0表示需要等待所有缓存刷新到磁盘后才可以关闭
    * 1表示只需将脏页数据刷新到磁盘就可以关闭，默认值
    * 2表示直接关闭，但是在下次启动时启用恢复程序
* innodb_force_recovery：指定恢复时到操作，有1-6个选项值
* innodb_version：查看innodb版本信息
* innodb_file_format：指定innodb文件格式
* innodb_read_ahead_threshold：指定一个区中的多少页被顺序访问时才触发线性预读取
* innodb_autoinc_lock_mode：设置插入自增长主键数据时的锁类型
    * Simple inserts简单插入操作：指插入之间就能知道确切的插入行数
    * Bulk inserts批量插入操作：指插入之间不知道确切的插入行数
    * Mixed-mode inserts混合插入模式：指有一部分主键是自增长的，有一部分主键是确定的
    * 0表示使用AUTO-INC Locking锁机制
    * 1表示针对Simple inserts使用互斥量进行累加主键值，对Bulk inserts使用AUTO-INC Locking锁机制
    * 2表示针对所有插入操作都使用互斥量进行累加主键值，并发时可能出现自增长的值不是连续的，因此恢复时要基于行数据恢复，不要基于语句恢复
* innodb_lock_wait_timeout：设置锁的阻塞等待时间，默认50s，避免死锁
* innodb_rollback_on_timeout：设置阻塞等待时间超时时是否回滚事务，默认关闭
* innodb_flush_log_at_trx_commit：设置重做日志缓存刷新策略
* innodb_support_xa：查询innodb是否开启XA事务
* 缓存池命中率计算：<img src='../images/mysql/缓存池命中率计算公式.png' width='300px'>